In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from collections import Counter

In [74]:
path_name = "data_booknlp/*.csv"
path_test = "data_test/*.csv"
N_PER = 10
N_token = 5

In [75]:
def get_df_PER(df):
    df_PER_1 = df[df['Tag'] == 'B-PER']
    df_PER_2 = df[df['Tag'] == 'S-PER']
    
    df_PER = pd.concat([df_PER_1, df_PER_2])
    df_PER = df_PER.sort_index(ascending=True)
    return df_PER

In [76]:
def get_counter(list_tag, N):
    list_elem_cleared = [elem for elem in list_tag if elem is not None]
    return Counter(list_elem_cleared).most_common(N)

In [77]:
def get_PER_surrounding(list_mention_token, list_head_token):
    list_surronding_PER = []
    for i in range(len(list_mention_token)):
        if list_mention_token[i] == 'B-PER' or list_mention_token[i] == 'S-PER':
            list_surronding_PER.append(list_head_token[i])
    return list_surronding_PER

In [91]:
def get_dict_mention_N_surronding_tokens(index, df_courant, N, name_seq):
    list_token = []
    str_base = ' '

    #list_mention_token = []
    #list_head_token = []
    dict_surronding_tokens = {}
    dict_surronding_tokens_main = {}

    for index_mention in index:
        if index_mention > N:
            for i in range(index_mention-N, index_mention+N):
                if i < len(df_courant):
                    if df_courant.loc[[i]]['Token'].values[0] is not None:
                        list_token.append(str(df_courant.loc[[i]]['Token'].values[0]))
                        #list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                        #list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])
        else:
            for i in range(index_mention, index_mention+N):
                if df_courant.loc[[i]]['Token'].values[0] is not None:
                    list_token.append(str(df_courant.loc[[i]]['Token'].values[0]))
                    #list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                    #list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])
        dict_surronding_tokens[name_seq+'_'+str(index_mention)] = str_base.join(list_token)
        dict_surronding_tokens_main.update(dict_surronding_tokens)
        list_token = []
        
    return dict_surronding_tokens_main

In [92]:
def get_dict_surronding_PER(df_courant, N_PER, N_token, doc_name):
    dict_surronding_tokens_main = {}
    df_PER = get_df_PER(df_courant)
    dict_mention = get_counter(list(df_PER['Head_tag']), N_PER)
    df_mention_roman = pd.DataFrame(dict_mention, columns=['mention', 'count'])
    df_mention_roman.dropna(inplace=True)
    list_mention = list(df_mention_roman['mention'])
    
    for i in range(0, len(list_mention)):# bug fixed (1-0)
        # get_index_mention w/ PER preds
        df_mentionned = df_courant[df_courant['Head_tag'] == list_mention[i]]
        index = df_mentionned.index
        # get N surronding tokens (useful for topic modelling, later), mention_tokens (BIES), head_mention_tokens (head coref mention)
        #list_surronding_tokens, list_surronding_mention_token, list_surronding_head_token = get_N_surronding_tokens(index, df_courant, N_token)
        # get head mention if mention is B or S-PER 
        #list_surronding_PER = get_PER_surrounding(list_surronding_mention_token, list_surronding_head_token)
        #dict_surronding_PER[list_mention[i]] = list_surronding_PER
        name_seq = doc_name+'_'+list_mention[i]
        dict_surronding_tokens = get_dict_mention_N_surronding_tokens(index, df_courant, N_token, name_seq)
        dict_surronding_tokens_main.update(dict_surronding_tokens)
        
    return dict_surronding_tokens_main, df_mention_roman  

In [93]:
def mouli_preds(path_name, N_PER, N_token):
    str_base = '_'
    for doc in tqdm(glob(path_name)):
        doc_name = doc.split('/')[1].split('.csv')[0]
        print(doc_name)
        df_courant = pd.read_csv(doc, sep = ',')        
        
        dict_surronding_PER, df_mention_PER = get_dict_surronding_PER(df_courant, N_PER, N_token, doc_name)
                
        df_mention_PER.to_csv(r'data/PER_novels/'+doc_name+'.csv',  encoding='utf-8', index=False)
        
        df_PER_surronding_tokens = pd.DataFrame.from_dict(dict_surronding_PER, orient='index', columns=['text_surronding'])
        df_PER_surronding_tokens.to_csv(r'data/PER_surronding_tokens/'+doc_name+'.csv',  encoding='utf-8', index=True)
        
        
    return df_PER_surronding_tokens

In [94]:
df_surronding_PER = mouli_preds(path_name, N_PER, N_token)

  0%|          | 0/2772 [00:00<?, ?it/s]

Glady-Alberic_Male-et-femelle
Belot-Adolphe_Le-drame-de-la-rue-de-la-paix
Mirabeau-Marie-Le-Harivel-de-Gonneville_Ctesse-de-Mirabeau-Jane-et-Germaine-Voyages-d-un-capitaine
Delly_Le-secret-de-la-Luzette
Beauvoir-Simone-de_La-force-des-choses
Rufin-Jean-Christophe_Rouge-Bresil
Carrere-Emmanuel_D-autres-vies-que-la-mienne
Dumas-Alexandre_Le-docteur-mysterieux
Sand-George_La-petite-Fadette
Gide-Andre_Voyage-au-Congo
Despentes-Virginie_Apocalypse-bebe
Mirbeau-Octave_Sebastien-Roch
Gaude-Laurent_Les-Oliviers-du-Negus
Chapus-Eugene_Les-soirees-de-Chantilly
Romains-Jules_Les-copains
Fabre-Ferdinand_Les-Courbezon
Gastyne-Jules-de_Le-lys-noir
Stolz-Madame-de_Violence-et-bonte
Mere-Elisabeth-Brossin-de_Le-Parc-aux-Cerfs-ou-Histoire-secrete-des-jeunes-demoiselles-qui-y-ont-ete-renfermees
Pergaud-Louis_De-Goupil-a-Margot
Rolin-Olivier_Bar-des-flots-noirs
Balzac-Honore-de_Maitre-Cornelius
Sue-Eugene_Le-morne-au-diable
Volodine-Antoine_Le-nom-des-singes
Tournier-Michel_Gilles-et-Jeanne
Bealu-Marcel_

Simenon-Georges_Mon-ami-Maigret
Chavannes-de-La-Giraudiere-Hippolyte-de_Les-Petits-voyageurs-en-Californie
Pigault-Lebrun_Mon-oncle-Thomas
Calet-Henri_Les-Grandes-Largeurs
Enard-Mathias_Boussole
Daeninckx-Didier_Lumiere-noire
Bergounioux-Pierre_B-17-G
Modiano-Patrick_La-Ronde-De-Nuit
Bergounioux-Pierre_Geologies
Courcy-Alfred-de_Le-Roman-cache
Leblanc-Maurice_Le-Bouchon-de-cristal
Vignon-Claude_Une-femme-romanesque-Adrien-Malaret-L-Exemple
Rolin-Dominique_Lettre-au-vieil-homme
Michon-Pierre_Vie-de-Joseph-Roulin
Balzac-Honore-de_Le-Requisitionnaire
Claudel-Philippe_L-Enquete
San-Antonio_Le-loup-habille-en-grand-mere
Dumas-Alexandre_Memoires-d-une-aveugle
Chavette-Eugene_La-conquete-d-une-cuisiniere
San-Antonio_Fleur-de-nave-vinaigrette
Jonquet-Thierry_la-bete-et-la-belle
Djian-Philippe_Impardonnables
Beigbeder-Frederic_Windows-on-the-World
Gavalda-Anna_L-echappee-belle
Balzac-Honore-de_Physiologie-du-mariage
Sarraute-Nathalie_disent-les-imbeciles
Greville-Henry_Les-epreuves-de-Raissa
Ey

Bourget-Paul_Laurence-Albani
Coppee-François_Henriette
Ivoi-Paul-d-_Corsaire-Triplex
Wey-Francis_Les-enfants-du-marquis-de-Ganges-ou-Les-expiations
de-Vigan-Delphine_Les-jolis-garçons
Merimee-Prosper_Carmen
Lesueur-Daniel_Levres-closes
Lamartine-Alphonse-de_Graziella
Gavalda-Anna_Ensemble-c-est-tout
Sand-George_Legendes-rustiques
Ernaux-Annie_Ce-qu-ils-disent-ou-rien
Pontmartin-Armand-de_Les-bruleurs-de-temples
Houssaye-Arsene_Les-Trois-duchesses
Erckmann-Chatrian_Contes-fantastiques
Fleuriot-Zenaide_Un-enfant-gate
Fourre-Maurice_La-Nuit-du-Rose-Hotel
Delly_Fleurs-du-foyer-fleurs-du-cloitre
Chevalier-H-Emile_La-fille-des-indiens-rouges
Cim-Albert_Histoire-d-un-baiser
Barbusse-Henri_Le-Feu
Verne-Jules_Le-Phare-du-Bout-du-Monde
Rodenbach-Georges_Bruges-la-Morte
Feval-Paul_La-Louve
Gustave-Le-Rouge-de-Brisay-Henri_L-Heroine-du-Colorado
Badin-Adolphe_Un-Parisien-chez-les-Russes
San-Antonio_San-Antonio-met-le-paquet
Daeninckx-Didier_Le-Geant-Inacheve
Guy-de-Maupassant_La-main-gauche
Sand-Ge

Loti-Pierre_Prime-jeunesse
Daudet-Alphonse_Jack
Barbey-d-Aurevilly-Jules_Le-Chevalier-des-Touches
Simenon-Georges_Le-voleur-de-Maigret
Bovet-Marie-Anne-de_Apres-le-divorce
Winckler-Martin_En-souvenir-d-Andre
Sirven-Alfred_La-Bigame-roman-contemporain
Sand-George_Monsieur-Sylvestre
Quignard-Pascal_Terrasse-a-Rome
Foenkinos-David_Le-potentiel-erotique-de-ma-femme
Simenon-Georges_Signe-Picpus
Marmette-Joseph_Chevalier-de-Mornac
Chevillard-Eric_Mourir-m-enrhume
Inconnu_Un-roman-sous-l-Empire
Leroux-Gaston_Les-etranges-noces-de-Rouletabille
Murail-Marie-Aude_En-nous-beaucoup-d-hommes-respirent
Druon-Maurice_Les-Rois-Maudits
Rasetti-Ernest_Les-Drames-du-village-Rosa-Romano
Tinseau-Leon-de_Plus-fort-que-la-haine
Blondin-Antoine_Quat-saisons
Ferney-Alice_Les-Autres
Blanchard-Pierre_Le-Nouvelliste-de-la-jeunesse
Rouaud-Jean_Un-peu-la-guerre
Simenon-Georges_Maigret-se-defend
Berthet-Elie_L-Incendiaire
Delly_Les-heures-de-la-vie
Vian-Boris_Et-on-tuera-tous-les-affreux
Dupuit-Albert_Pauline-Tardiv

...

Ernaux-Annie_La-place
Delly_Le-chant-de-la-misere
Delly_Ourida
Cladel-Leon_Ompdrailles-le-Tombeau-des-lutteurs
Morin-Louis_Grand-mere-avait-des-defauts-
Allais-Alphonse_Le-parapluie-de-l-escouade
Echenoz-Jean_14
Guise-Charles-de_Le-Cap-au-Diable
Stolz-Madame-de_La-Maison-roulante
Pagnol-Marcel_La-Gloire-de-mon-pere
Proust-Marcel_La-Prisonniere-A-la-recherche-du-temps-perdu
Nemirovsky-Irene_Les-Feux-de-l-automne
Colomb-Josephine_Chloris-et-Jeanneton
Leblanc-Maurice_La-demoiselle-aux-yeux-verts
San-Antonio_Du-brut-pour-les-brutes
Barthelemy-Charles_Histoire-de-la-Bretagne-ancienne-et-moderne
Achard-Amedee_Madame-Rose
Veuzit-Max-du_La-chataigneraie
Valles-Jules_L-Insurge
Simenon-Georges_Maigret-a-Vichy
Modiano-Patrick_Chien-de-printemps
San-Antonio_La-tombola-des-voyous
Vigan-Delphine-de_Les-heures-souterraines
Bouraoui-Nina_Garcon-Manque
Delly_Les-hiboux-des-roches-rouges
Nothomb-Amelie_Ni-d-Eve-ni-d-Adam
San-Antonio_Rue-des-macchabees
San-Antonio_Passez-moi-la-Joconde
Loti-Pierre_Les-De

Bazin-Rene_De-toute-son-ame
Philippe-Charles-Louis_La-mere-et-l-enfant
Bouvier-Alexis_Iza-Lolotte-et-Compagnie
Modiano-Patrick_Livret-de-famille
Balzac-Honore-de_Le-Lys-dans-la-vallee
Veuzit-Max-du_Mirages
Volodine-Antoine_Nos-animaux-preferes
Chateaubriand-François-Rene-de_Memoires-d-Outre-Tombe
Sagan-Françoise_Un-peu-de-soleil-dans-l-eau-froide
San-Antonio_Faut-il-vous-l-envelopper
Zaccone-Pierre_Le-Courrier-de-Lyon
Sand-George_Les-maitres-sonneurs
Le-Rouge-Gustave_La-Guerre-des-vampires
Dumas-Alexandre_La-jeunesse-de-Pierrot
Riboulet-Mathieu_L-Amant-Des-Morts
San-Antonio_Tout-le-plaisir-est-pour-moi
Villiers-de-l-Isle-Adam-Auguste_Tribulat-Bonhomet
Lesueur-Daniel_Passion-slave
Vie-Anduze-Henri_Dernier-amour-de-Cinq-Mars-roman-historique
Delly_Le-secret-du-Kou-Kou-Noor
Guilloux-Louis_Le-pain-des-reves
Gerard-Jules_La-Chasse-au-lion
Nothomb-Amelie_Cosmetique-de-l-ennemi
Simonin-Albert_Le-Hotu
Gary-Romain_La-Tete-coupable
Sorr-Angelo-de_Fantome-de-la-rue-de-Venise
Malot-Hector_Baccara


Aimard-Gustave_Le-Chasseur-de-rats
Leblanc-Maurice_Le-Cabochon-d-emeraude
Aimard-Gustave_Les-trappeurs-de-l-Arkansas
Segur-comtesse-de_Les-Memoires-d-un-ane
Bergounioux-Pierre_La-fin-du-monde-en-avançant
Leblanc-Maurice_La-dent-d-Hercule-Petitgris
Tavard-Eugene_Les-Effets-et-les-causes-histoire-lamentable-echafaudee-sur-une-queue-de-billard
Dumas-Alexandre_Les-Louves-de-Machecoul
Le-Roy-Albert_L-Amour-sans-phrase
Des-Forets-Rene-Louis_Ostinato
Rouaud-Jean_La-femme-promise
Merimee-Prosper_La-Venus-d-Ille
Graffigny-H-de_Le-tour-de-France-en-aeroplane
San-Antonio_Tango-chinetoque
Balzac-Honore-de_Le-Cabinet-des-Antiques
Claretie-Jules_Bouddha
Delly_Sous-le-masque
Bourget-Paul_La-duchesse-bleue
Begaudeau-François_Molecules
Musso-Guillaume_L-appel-de-l-ange
Simon-Claude_Triptyque
Martin-du-Gard-Roger_Les-Thibault
Rolland-Romain_L-ame-enchantee-Annette-et-Sylvie
Leblanc-Maurice_L-Eclat-d-obus
Lermina-Jules_L-elixir-de-vie
Noir-Louis_Une-chasse-a-courre-au-Pole-nord
Musso-Guillaume_Parce-que-

Lermina-Jules_To-Ho-Le-Tueur-d-or
Witt-Henriette-de_Une-soeur
Dhanys-Marcel_Le-voeu-de-Phanette
Zevaco-Michel_L-epopee-d-amour
Decourcelle-Pierre_Mam-zelle-Misere
Belot-Adolphe_Le-secret-terrible--memoires-d-un-caissier
Rolin-Olivier_Port-Soudan
Le-Rouge-Gustave_Cinq-nouvelles-extraordinaires
Loti-Pierre_Pecheur-d-Islande
Yourcenar-Marguerite_Archives-du-Nord
Fleuriot-Zenaide_Bonasse
France-Anatole_L
Calonne-Alphonse-de_La-Ferme-des-moines
Millet-Richard_Lauve-le-pur
Gondry-Du-Jardinet-Jules_Sur-le-bucher-ou-le-sort-des-femmes
Scholl-Aurelien_Les-Gens-tares
Moselly-Emile_Terres-lorraines
Sue-Eugene_Atar-Gull
Greville-Henry_Dosia
Verne-Jules_Le-Sphinx-des-Glaces
Zola-Emile_La-terre
Sand-George_Journal-d-un-voyageur-pendant-la-guerre
Delly_La-biche-au-bois
Girardin-Delphine-de_La-canne-de-M-de-Balzac
Ernaux-Annie_Les-annees
Lemay-Pamphile_Picounoc-le-maudit
Ivoi-Paul-d-_Le-Sergent-Simplet-a-travers-les-colonies-françaises
Perec-Georges_L-Attentat-de-Sarajevo
Rouaud-Jean_La-fiancee-juive


Bazin-Rene_Madame-Corentine
Sue-Eugene_Miss-Mary-ou-l-Institutrice
Balzac-Honore-de_Un-drame-au-bord-de-la-mer
Berthoud-Henry_Les-Filleules-de-Rubens
Dhotel-Andre_L-Enfant-qui-disait-n-importe-quoi
Modiano-Patrick_La-Petite-Bijou
Theuriet-Andre_Andre-Theuriet-La-Maison-des-Deux-Barbeaux-Le-Sang-des-Finoel
Sand-George_Indiana
Loti-Pierre_L-exilee
Djian-Philippe_Ça-c-est-un-baiser
Troyat-Henri_Les-Dames-de-Siberie
Nemirovsky-Irene_Les-Mouches-d-automne
San-Antonio_Bas-les-pattes
Djian-Philippe_Love-Song
Balzac-Honore-de_Les-Chouans-ou-La-Bretagne-en-1799
San-Antonio_Ca-ne-s-invente-pas
Delly_La-porte-scellee
Bouvier-Alexis_Le-Bel-Alphonse
Ulbach-Louis_L-ile-des-reves
Arlincourt-Charles-Victor_Le-renégat
Loti-Pierre_Un-pelerin-d-Angkor
Dumas-Alexandre_Gabriel-Lambert
Bazin-Rene_Donatienne
Nothomb-Amelie_Une-forme-de-vie
Leger-Nathalie_Supplement-a-la-vie-de-Barbara-Loden
Veuzit-Max-du_Le-vieux-puits
Valles-Jules_Les-Refractaires
Modiano-Patrick_L-herbe-des-nuits
Valles-Jules_L-Enfant
Cohe

In [95]:
df_surronding_PER.head()

,text_surronding
Boisgobey-Fortune-du_Le-crime-de-l-Opera_<la locataire absente-2352>_57179,", quand on vit apparaître la locataire absente..."
Boisgobey-Fortune-du_Le-crime-de-l-Opera_<la locataire absente-2352>_57184,la locataire absente ; mais elle avait toujour...
Boisgobey-Fortune-du_Le-crime-de-l-Opera_<la locataire absente-2352>_57201,"gens , qu' on ne lui fit pas mauvais accueil"
Boisgobey-Fortune-du_Le-crime-de-l-Opera_<la locataire absente-2352>_57211,", et qu' on ne lui adressa pas trop de"
Boisgobey-Fortune-du_Le-crime-de-l-Opera_<la locataire absente-2352>_57250,dans un bel équipage demander mademoiselle Les...
